In [1]:
#!pip install git+https://github.com/openai/whisper.git@248b6cb124225dd263bb9bd32d060b6517e067f8

In [2]:
#!pip install accelerate -U

In [3]:
#important
#!pip install librosa==0.9.2 

In [4]:
#!pip install scipy
#!pip install datasets>=2.6.1
#!pip install git+https://github.com/huggingface/transformers
#!pip install librosa
#!pip install evaluate>=0.30
#!pip install jiwer
#!pip install gradio
#!pip install transformers[torch]
#!pip install transformers

In [5]:
import whisper 
import pandas as pd 
import numpy as np
import torch 
from collections import OrderedDict

In [6]:
import pickle
# Load the list from the file
with open("/home/projects/dharel/eranbe/Eran/all_data_list_poc_segprotfoc_include21.pkl", 'rb') as f:
    tal_alona_pkl_best_data = pickle.load(f)


In [7]:
# order matter!

all_data_list = tal_alona_pkl_best_data

In [8]:
import pandas as pd
import numpy as np

In [9]:
df = pd.DataFrame(all_data_list)

In [10]:
for index, row in df.iterrows():
    row['audio']['array'] = np.array(row['audio']['array'], dtype=np.float32)


In [11]:
from datasets import Dataset, DatasetDict


seed = 123  # you can choose any integer seed you like

train_df = df.iloc[:int(0.95*len(df))] # very important line of code, in order for not mix test with train
train_df = train_df.sample(frac=1, random_state=seed).reset_index(drop=True)  # deterministic shuffle and reset index
train_dataset = Dataset.from_pandas(train_df)


# Create the Dataset objects
#train_dataset = Dataset.from_pandas(df.iloc[:int(0.95*len(df))])




test_dataset = Dataset.from_pandas(df.iloc[int(0.95*len(df)):])

# Create the DatasetDict object
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


In [12]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1168
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 62
    })
})

In [13]:
path = '/home/projects/dharel/eranbe/Eran/whisper_seg_focus_prototype_wexac_22_07_2023_v2booya_offline/checkpoint-4000'

In [14]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained(path)

In [15]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(path, language="english", task="transcribe")

In [16]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(path, language="english", task="transcribe")

In [17]:
#need to trancate sequence ,model works only with 1024 token.
#@@@@@@@@@@
#@@@@@@@@@@
#$$$$$$$$$$$$$$$$$
#MAX_LENGTH = 1024
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # truncate target text to a maximum length
    #sentence = batch["sentence"][:MAX_LENGTH]
    sentence = batch["sentence"]


    """

    # split the string into list of words
    words = sentence.split()

    # define the markers
    markers = ['TRUE1TRUE', 'FALSE1TRUE', 'TRUE2TRUE', 'FALSE2TRUE', 'TRUE3TRUE', 'FALSE3TRUE', 'TRUE1FALSE', 'FALSE1FALSE', 'TRUE2FALSE', 'FALSE2FALSE', 'TRUE3FALSE', 'FALSE3FALSE']

    # process the words
    output_words = [word.lower() if word not in markers else word for word in words]

    # join the words back into a string
    sentence = ' '.join(output_words)

    """





    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    # encode target text to label ids 
    labels_ids = tokenizer(sentence).input_ids
    # truncate labels to a maximum length
    #labels_ids = labels_ids[:MAX_LENGTH]
    batch["labels"] = labels_ids
    return batch

In [18]:
dataset = dataset_dict.map(prepare_dataset, remove_columns=dataset_dict.column_names["train"], num_proc=8)

Map (num_proc=8):   0%|          | 0/1168 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (6891 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2643 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3800 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1704 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4523 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=8):   0%|          | 0/62 [00:00<?, ? examples/s]

In [19]:
max_input_length = 30
min_input_length = 0


def is_audio_in_length_range(length):
    return length > min_input_length and length <= max_input_length

In [20]:
vectorized_datasets = dataset.filter(
    is_audio_in_length_range, num_proc=1, input_columns=["input_length"]
)

vectorized_datasets

Filter:   0%|          | 0/1168 [00:00<?, ? examples/s]

Filter:   0%|          | 0/62 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 1134
    })
    test: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 61
    })
})

In [21]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(path)

In [22]:
model.config.max_length

448

In [23]:
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")

model.config.suppress_tokens = []

In [24]:
max_label_length = model.config.max_length
max_label_length

448

In [25]:
def is_labels_in_length_range(labels):
    return len(labels) < max_label_length

In [26]:
vectorized_datasets = vectorized_datasets.filter(
    is_labels_in_length_range, num_proc=1, input_columns=["labels"]
)

vectorized_datasets

Filter:   0%|          | 0/1134 [00:00<?, ? examples/s]

Filter:   0%|          | 0/61 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 1131
    })
    test: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 61
    })
})

In [27]:
common_voice = vectorized_datasets

In [28]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        print(batch)
        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        print(labels)
        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [29]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [30]:
import evaluate

metric = evaluate.load("wer")

In [31]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [32]:
from transformers import pipeline

#pipe = pipeline('automatic-speech-recognition',"/content/drive/MyDrive/Weizmann/whisper_focus_segmentation/FOCUS_SEGMENTATION_WITH_DATAISSUE/checkpoint-3000",chunk_length_s=30)

In [33]:
import gradio as gr
import librosa

In [34]:
from datetime import datetime
from pydub import AudioSegment
import os



def transcribe(audio):
    y, sr = librosa.load(audio)
    audio = librosa.resample(y, sr, 16000)
    #text = pipe(audio.flatten())["text"]
    #return transform_2_df(text)#bold_words(transform_to_prototype(transform_2_df(text)))

In [35]:
import torch

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
hf_key_change = pd.read_csv(f"/home/projects/dharel/eranbe/Eran/model_parameters_key_change_from_hf_to_wisper.csv")
hf_key_change = hf_key_change.assign(hagginface_w_prefix = "model." + hf_key_change.hagginface)
hf_key_change_dict = hf_key_change.set_index("hagginface_w_prefix")["whisper"].to_dict()
#hf_key_change_dict = hf_key_change.set_index("hagginface")["whisper"].to_dict()
state_dict = torch.load(path+"/pytorch_model.bin",map_location=device)


In [38]:
#state_dict_correct = OrderedDict([(hf_key_change_dict[k],v) for k,v in state_dict.items()])

In [39]:
#state_dict_correct = OrderedDict([(hf_key_change_dict[k],v) for k,v in state_dict.items()])
state_dict_correct = OrderedDict([(hf_key_change_dict[k],v) for k,v in state_dict.items() if k != 'proj_out.weight'])

In [40]:
model = whisper.load_model("large-v2",download_root="/home/projects/dharel/eranbe/Eran/whisper-large-base",device=device)
#model = whisper.load_model("large",download_root="/whisper-large-base",device=device)


In [41]:
model.load_state_dict(state_dict_correct)

<All keys matched successfully>

In [42]:
model.eval()

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1280, out_features=1280, bias=True)
          (key): Linear(in_features=1280, out_features=1280, bias=False)
          (value): Linear(in_features=1280, out_features=1280, bias=True)
          (out): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1280, out_features=5120, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (mlp_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((

In [43]:
def get_whisper_tagging(transcription, audio_path, model):
    """
    This methods applied the next tactic for getting prediction for each word in the transcript:
     - t <- initial tokens
     - for each word in the transcript:
      - predict the next token for the last token in t
      - repeat previous step three times to get TRUE/FALSE + 1/2/3 + TRUE/FALSE
      - add the word tokens to t
      - update dataframe with the added word and the generated tags
    @param transcription: lower case words seperated by spaces
    @param audio_features: features generated from the audio matching the transcript
    @param model: the whisper model
    @return: df with prediction for each word in the transcript, string output of the model
    """

    mel = whisper.audio.log_mel_spectrogram(audio_path)
    mel = whisper.audio.pad_or_trim(mel, length=whisper.audio.N_FRAMES)
    with torch.no_grad():
        audio_features = model.encoder(mel.unsqueeze(0).to(model.device))

    # getting tokenizer
    tokenizer = whisper.tokenizer.get_tokenizer(True)
    device = model.device

    transcription_parts_wo_spaces = [p for p in transcription.split(" ")] 
    transcription_parts = [" " + p for p in transcription_parts_wo_spaces] # " w1" + " w2" + " w3"
    transcription_parts_tokens = [tokenizer.encode(p) for p in transcription_parts]

    starting_tokens = [50258, 50259, 50359, 50364]  # '<|startoftranscript|><|en|><|transcribe|>'. last token is ''

    res_segmentation = []
    res_prototype = []
    res_emphasis = []

    tokens = starting_tokens
    with torch.no_grad():
        for i in range(len(transcription_parts_tokens)):
            ### Segmentation running the model to figure out segmentation
            tokens_tensor = torch.Tensor([tokens]).int().to(device)
            
            logits_for_transcript = model.decoder(tokens_tensor, audio_features)
            logits_for_transcript = logits_for_transcript.detach().cpu()
            logits_for_transcript = logits_for_transcript[:, -1]
            # filters - everything is -inf other than ' TR',' F','TR','F'
            tokens_to_choose_from = tokenizer.encode(" TR FTRF") 
            tokens_to_choose = logits_for_transcript[:, tokens_to_choose_from]
            logits_for_transcript[:, :] = -np.inf
            logits_for_transcript[:, tokens_to_choose_from] = tokens_to_choose

            best_tokens = logits_for_transcript.argmax(-1, True)

            best_tokens_int = best_tokens.cpu().detach().numpy()[0][0]

            tokens = tokens + [best_tokens_int]
            if tokens[-1] in tokenizer.encode(" TRTR"):
                tokens = tokens + tokenizer.encode("UE")
                res_segmentation.append(True)
            else:
                tokens = tokens + tokenizer.encode('ALSE')
                res_segmentation.append(False)




            ### Prototype running the model to figure out prototype
            tokens_tensor = torch.Tensor([tokens]).int().to(device)
            
            logits_for_transcript = model.decoder(tokens_tensor, audio_features)
            logits_for_transcript = logits_for_transcript.detach().cpu()
            logits_for_transcript = logits_for_transcript[:, -1]
            # filters - everything is -inf other than '1','2'
            tokens_to_choose_from = tokenizer.encode("1") + tokenizer.encode("2") + tokenizer.encode("3")
            tokens_to_choose = logits_for_transcript[:, tokens_to_choose_from]
            logits_for_transcript[:, :] = -np.inf
            logits_for_transcript[:, tokens_to_choose_from] = tokens_to_choose

            best_tokens = logits_for_transcript.argmax(-1, True)
            best_tokens_int = best_tokens.cpu().detach().numpy()[0][0]
            res_prototype.append(tokenizer.decode((best_tokens_int,))[0])

            tokens = tokens + [best_tokens_int]










            

            ### Focus running the model to figure out emphasis
            tokens_tensor = torch.Tensor([tokens]).int().to(device)
            
            logits_for_transcript = model.decoder(tokens_tensor, audio_features)
            logits_for_transcript = logits_for_transcript.detach().cpu()
            logits_for_transcript = logits_for_transcript[:, -1]
            # filters - everything is -inf other than ' TR',' F','TR','F'
            tokens_to_choose_from = tokenizer.encode(" TR FTRF")
            tokens_to_choose = logits_for_transcript[:, tokens_to_choose_from]
            logits_for_transcript[:, :] = -np.inf
            logits_for_transcript[:, tokens_to_choose_from] = tokens_to_choose

            best_tokens = logits_for_transcript.argmax(-1, True)

            best_tokens_int = best_tokens.cpu().detach().numpy()[0][0]

            tokens = tokens + [best_tokens_int]
            if tokens[-1] in tokenizer.encode(" TRTR"):
                tokens = tokens + tokenizer.encode("UE")
                res_emphasis.append(True)
            else:
                tokens = tokens + tokenizer.encode('ALSE')
                res_emphasis.append(False)

            ### Adding next word token
            tokens = tokens + transcription_parts_tokens[i]




   


            

    expected_regular_model_result = tokenizer.decode(
        tokens[len(starting_tokens):]
    )

    result_df = pd.DataFrame({
        "word": transcription_parts_wo_spaces,
        "segmentation": res_segmentation ,
        "prototype": res_prototype,
        "emphasis": res_emphasis, 
    })
    return result_df, expected_regular_model_result



In [44]:


def get_mem():
    """
    returns the amount of free memory in cuda in GB
    """
    # t = torch.cuda.get_device_properties(0).total_memory
    r = torch.cuda.memory_reserved(0)
    a = torch.cuda.memory_allocated(0)
    f = r - a  # free inside reserved
    return f / 1024 ** 3


def get_audio_features_from_mel(mel, model):
    mel = whisper.audio.pad_or_trim(mel,length=whisper.audio.N_FRAMES)
    with torch.no_grad():
        audio_features = model.encoder(mel.unsqueeze(0).to(model.device))
    # n_group = 1  # beam search parameter (how many candidates to keep track of?)
    # audio_features = audio_features.repeat_interleave(n_group, dim=0)
    return audio_features



In [45]:
import pandas as pd
import numpy as np

def transform_2_df(orig_df):
    new_df_list = []

    for _, row in orig_df.iterrows():
        sentence = row['sentence']
        audio_array = row['audio_array']

        words = []
        flags = []

        for token in sentence.split():
            if token.startswith("TRUE") or token.startswith("FALSE"):
                flags.append(token)
            else:
                words.append(token)

        min_length = min(len(words), len(flags))

        data = {"FOCUS_SEGMENTATION_PROTOTYPE": flags[:min_length]}
        df = pd.DataFrame(data)

        # add the words column
        df['WORDS'] = words[:min_length]

        # add the audio array column
        df['ARRAY'] = [audio_array] * min_length

        # add the index column
        df['INDEX'] = [row.name] * min_length

        # append the DataFrame to the list
        new_df_list.append(df[['FOCUS_SEGMENTATION_PROTOTYPE', 'WORDS', 'ARRAY', 'INDEX']])

    # concatenate all the DataFrames in the list into a single DataFrame
    new_df = pd.concat(new_df_list, ignore_index=True)

    return new_df

# Updated function


def transform_2_df2(input_str):
    words = []
    flags = []

    for token in input_str.split():
        if token.startswith("TRUE") or token.startswith("FALSE"):
            flags.append(token)
        else:
            words.append(token)

    min_length = min(len(words), len(flags))

    data = {"col2": flags[:min_length]}
    df = pd.DataFrame(data)

    # split the second column
    df[['SEGMENTATION', 'PROTOTYPE','FOCUS']] = df['col2'].str.extract('(TRUE|FALSE)(\d)(TRUE|FALSE)', expand=True)
    
    # add the words column
    df['WORDS'] = words[:min_length]

    return df[['SEGMENTATION','PROTOTYPE','FOCUS', 'WORDS']]



In [46]:
VALIDATION_DF = df.iloc[int(0.95*len(df)):] 

In [47]:
# Assuming df is your DataFrame and 'audio' is your column.
VALIDATION_DF['audio_array'] = VALIDATION_DF['audio'].apply(lambda x: x['array'])


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [48]:
VALIDATION_DF.head()

,audio,sentence,audio_array
1168,{'path': '/home/eranbe/dharelg/datasets/whispe...,TRUE3FALSE And FALSE3FALSE the FALSE3TRUE medi...,"[0.00070413994, 0.00093696994, 0.00059676066, ..."
1169,{'path': '/home/eranbe/dharelg/datasets/whispe...,TRUE2TRUE Yeah TRUE3FALSE The FALSE3TRUE attac...,"[-0.0011175656, -0.0015582022, -0.0015195274, ..."
1170,{'path': '/home/eranbe/dharelg/datasets/whispe...,TRUE2FALSE long FALSE2TRUE time TRUE3FALSE And...,"[0.036355555, 0.04650424, 0.023045568, 0.01195..."
1171,{'path': '/home/eranbe/dharelg/datasets/whispe...,TRUE2TRUE I FALSE2FALSE don't FALSE2FALSE have...,"[-0.09080503, -0.14662455, -0.14236635, -0.168..."
1172,{'path': '/home/eranbe/dharelg/datasets/whispe...,TRUE3TRUE Frank TRUE1TRUE once FALSE1FALSE thi...,"[-0.0006250964, -0.0011925529, -0.0017315487, ..."


In [49]:
from sklearn.metrics import confusion_matrix

In [50]:
def create_example_df_pred(INDEX,VALIDATION = VALIDATION_DF):
  # Assuming df is your DataFrame and 'audio' is your column.
  VALIDATION_DF['audio_array'] = VALIDATION_DF['audio'].apply(lambda x: x['array'])
  df2 = transform_2_df(VALIDATION_DF)
  transcript1178 = " ".join(df2[df2.INDEX==INDEX].WORDS)
  path_np1178 = df2[df2.INDEX==INDEX].ARRAY.iloc[0]
  res_df, res_str = get_whisper_tagging(transcript1178,path_np1178,model)
  df_true = transform_2_df2(VALIDATION_DF.loc[INDEX].sentence)
  df_pred = res_df
  merge_result = pd.merge(df_true,df_pred,left_index = True, right_index = True)
  merge_result_rename = merge_result.rename(columns={
      'FOCUS': 'focus_true',
      'SEGMENTATION': 'segmentation_true',
      'word': 'word_pred',
      'WORDS': 'word_true',
      'segmentation': 'segmentation_pred',
      'emphasis': 'focus_pred',
      'PROTOTYPE': 'prototype_true',
      'prototype': 'prototype_pred'
  })

  merge_result_rename_reordered = merge_result_rename[['focus_true','focus_pred','segmentation_true','segmentation_pred','prototype_true','prototype_pred','word_true','word_pred']]
  return merge_result_rename_reordered

In [51]:
VALIDATION_DF.index

RangeIndex(start=1168, stop=1230, step=1)

In [52]:
"""
turns_number_list = [1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130,
       1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141,
       1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152,
       1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163,
       1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174,
       1175, 1176, 1177, 1178]
"""

'\nturns_number_list = [1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130,\n       1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141,\n       1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152,\n       1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163,\n       1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174,\n       1175, 1176, 1177, 1178]\n'

In [53]:
#len(turns_number_list)

In [54]:
pred_dfs = []
for num in VALIDATION_DF.index:
  pred_dfs.append(create_example_df_pred(num))



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [55]:
pred_df = pd.concat(pred_dfs,axis=0)

In [56]:
# Convert to boolean
pred_df['focus_true'] = pred_df['focus_true'].map({'TRUE': True, 'FALSE': False})
# Convert to boolean
pred_df['segmentation_true'] = pred_df['segmentation_true'].map({'TRUE': True, 'FALSE': False})


In [57]:
conf_matrix = confusion_matrix(pred_df['focus_true'], pred_df['focus_pred'])
print(conf_matrix)

[[658 108]
 [ 88 152]]


In [58]:
from sklearn import metrics

In [59]:
metrics.cohen_kappa_score(pred_df['focus_true'], pred_df['focus_pred'])

0.47864621893178205

In [60]:
metrics.cohen_kappa_score(pred_df['segmentation_true'], pred_df['segmentation_pred'])

0.934343132189378

In [61]:
metrics.cohen_kappa_score(pred_df['prototype_true'], pred_df['prototype_pred'])

0.6484800970581035

In [62]:
# seems like 3000 itereations improve results.

In [63]:
from sklearn.metrics import accuracy_score

In [64]:
accuracy_score(pred_df['segmentation_true'], pred_df['segmentation_pred'])

0.9761431411530815

In [65]:
accuracy_score(pred_df['focus_true'], pred_df['focus_pred'])

0.805168986083499

# metric for evaluating Prototype

In [66]:
#https://pypi.org/project/nervaluate/

In [67]:
pip install nervaluate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [68]:
import nervaluate
from nervaluate import Evaluator

In [69]:
#example


true = [
    [{"label": "PER", "start": 2, "end": 4}],
    [{"label": "LOC", "start": 1, "end": 2},
     {"label": "LOC", "start": 3, "end": 4}]
]

pred = [
    [{"label": "PER", "start": 2, "end": 4}],
    [{"label": "LOC", "start": 1, "end": 2},
     {"label": "LOC", "start": 3, "end": 4}]
]

from nervaluate import Evaluator

evaluator = Evaluator(true, pred, tags=['LOC', 'PER'])

# Returns overall metrics and metrics for each tag

results, results_per_tag = evaluator.evaluate()

print(results)

{'ent_type': {'correct': 3, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 3, 'actual': 3, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'partial': {'correct': 3, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 3, 'actual': 3, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'strict': {'correct': 3, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 3, 'actual': 3, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'exact': {'correct': 3, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 3, 'actual': 3, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}


In [70]:
results_per_tag

{'LOC': {'ent_type': {'correct': 2,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurious': 0,
   'possible': 2,
   'actual': 2,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  'partial': {'correct': 2,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurious': 0,
   'possible': 2,
   'actual': 2,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  'strict': {'correct': 2,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurious': 0,
   'possible': 2,
   'actual': 2,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  'exact': {'correct': 2,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurious': 0,
   'possible': 2,
   'actual': 2,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0}},
 'PER': {'ent_type': {'correct': 1,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurious': 0,
   'possible': 1,
   'actual': 1,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  'partial': {'correct': 1,
   'incorrect': 0,
   'parti

In [71]:
from typing import List, Dict

def consecutive_numbers(lst: List[int]) -> List[Dict[str, int]]:
    """
    This function takes a list of numbers and returns a list of dictionaries.
    Each dictionary represents a sequence of the same number in the input list.
    The dictionary contains the number (flag), the start index, and the end index of the sequence.

    :param lst: List of integers
    :return: List of dictionaries representing sequences of the same number
    """
    # Initialize empty result list
    result = []

    # Initialize start_index and current_flag to None
    start_index = None
    current_flag = None

    # Iterate over the list with index
    for i, item in enumerate(lst):
        if start_index is None:  # Check if start_index is None, this indicates the start of a sequence
            start_index = i
            current_flag = item
        elif item != current_flag:  # Check if the item is not equal to current_flag, this indicates the end of a sequence
            # Add the sequence (start_index to i-1) to the result list
            result.append({
                "label": str(current_flag),
                "start": start_index,
                "end": i - 1
            })
            start_index = i  # Set the current index as the start_index for the next sequence
            current_flag = item  # Set the current item as the flag for the next sequence

    # After the loop, check if there is an unfinished sequence. If yes, add it to the result.
    if start_index is not None:
        result.append({
            "label": str(current_flag),
            "start": start_index,
            "end": len(lst) - 1  # The end index of the last sequence is the last index of the list
        })

    return result



In [72]:
test = [1, 1, 1, 2, 2, 2, 2]

In [73]:
print(consecutive_numbers(test))

[{'label': '1', 'start': 0, 'end': 2}, {'label': '2', 'start': 3, 'end': 6}]


In [74]:
true =  [consecutive_numbers(list(pred_df['prototype_true']))]

In [75]:
pred = [consecutive_numbers(list(pred_df['prototype_pred']))]

In [76]:
true[0][:5]

[{'label': '3', 'start': 0, 'end': 4},
 {'label': '2', 'start': 5, 'end': 11},
 {'label': '3', 'start': 12, 'end': 20},
 {'label': '2', 'start': 21, 'end': 22},
 {'label': '3', 'start': 23, 'end': 26}]

In [77]:
evaluator = Evaluator(true, pred, tags=['1', '2', '3'])

In [78]:
evaluator

In [79]:
 results , results_per_tag = evaluator.evaluate()

In [80]:
results_per_tag['1']['ent_type']

{'correct': 7,
 'incorrect': 2,
 'partial': 0,
 'missed': 1,
 'spurious': 0,
 'possible': 10,
 'actual': 9,
 'precision': 0.7777777777777778,
 'recall': 0.7,
 'f1': 0.7368421052631577}

In [81]:
results_per_tag['2']['ent_type']

{'correct': 35,
 'incorrect': 6,
 'partial': 0,
 'missed': 10,
 'spurious': 10,
 'possible': 51,
 'actual': 51,
 'precision': 0.6862745098039216,
 'recall': 0.6862745098039216,
 'f1': 0.6862745098039216}

In [82]:
results_per_tag['3']['ent_type']

{'correct': 42,
 'incorrect': 4,
 'partial': 0,
 'missed': 8,
 'spurious': 6,
 'possible': 54,
 'actual': 52,
 'precision': 0.8076923076923077,
 'recall': 0.7777777777777778,
 'f1': 0.7924528301886792}

In [83]:
results['ent_type']

{'correct': 84,
 'incorrect': 12,
 'partial': 0,
 'missed': 19,
 'spurious': 16,
 'possible': 115,
 'actual': 112,
 'precision': 0.75,
 'recall': 0.7304347826086957,
 'f1': 0.7400881057268722}

In [84]:
list(pred_df['prototype_true'])[:5]

['3', '3', '3', '3', '3']

In [85]:
results

{'ent_type': {'correct': 84,
  'incorrect': 12,
  'partial': 0,
  'missed': 19,
  'spurious': 16,
  'possible': 115,
  'actual': 112,
  'precision': 0.75,
  'recall': 0.7304347826086957,
  'f1': 0.7400881057268722},
 'partial': {'correct': 42,
  'incorrect': 0,
  'partial': 54,
  'missed': 19,
  'spurious': 16,
  'possible': 115,
  'actual': 112,
  'precision': 0.6160714285714286,
  'recall': 0.6,
  'f1': 0.6079295154185023},
 'strict': {'correct': 42,
  'incorrect': 54,
  'partial': 0,
  'missed': 19,
  'spurious': 16,
  'possible': 115,
  'actual': 112,
  'precision': 0.375,
  'recall': 0.3652173913043478,
  'f1': 0.3700440528634361},
 'exact': {'correct': 42,
  'incorrect': 54,
  'partial': 0,
  'missed': 19,
  'spurious': 16,
  'possible': 115,
  'actual': 112,
  'precision': 0.375,
  'recall': 0.3652173913043478,
  'f1': 0.3700440528634361}}